In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd 

# 1. Data preparation for model training/testing

In [2]:
data_v1 = pd.read_csv('./PIB_all_July_10_2019.csv') #Load the whole dataset
data_v2 = data_v1.copy()
data_v3 = data_v1.copy()

In [44]:
# storing by visit version
list_v1 = []
for i in range(1,10):
    list_v1.append('pdt0000'+str(i))
for i in range(10,100):
    list_v1.append('pdt000'+str(i))
for i in range(100,260):
    list_v1.append('pdt00'+str(i))
    

list_v2 = []
for i in range(1,10):
    list_v2.append('pdt0000'+str(i)+'_v2')
for i in range(10,100):
    list_v2.append('pdt000'+str(i)+'_v2')
for i in range(100,260):
    list_v2.append('pdt00'+str(i)+'_v2')
list_v3 = []
for i in range(1,10):
    list_v3.append('pdt0000'+str(i)+'_v3')
for i in range(10,100):
    list_v3.append('pdt000'+str(i)+'_v3')
for i in range(100,260):
    list_v3.append('pdt00'+str(i)+'_v3')

In [5]:
#list_v1

In [45]:
#Cleaning, leaving subjects only present in all 3 visits
delete_list_1 = []
delete_list_2 = []
delete_list_3 = []
for i in range(0,data_v1.shape[0]):
    if not(data_v1['subjectid'][i] in list_v1):
        delete_list_1.append(i)
    if not(data_v2['subjectid'][i] in list_v2):
        delete_list_2.append(i)
    if not(data_v3['subjectid'][i] in list_v3):
        delete_list_3.append(i)

In [46]:
data_v1.drop(delete_list_1 ,inplace=True)
data_v2.drop(delete_list_2 ,inplace=True)
data_v3.drop(delete_list_3 ,inplace=True)
data_v1.reset_index(drop=True, inplace=True)
data_v2.reset_index(drop=True, inplace=True)
data_v3.reset_index(drop=True, inplace=True)

In [47]:
delete_list = []
list_v1 = data_v1['subjectid'].values.tolist()
list_v2 = data_v2['subjectid'].values.tolist()
list_v3 = data_v3['subjectid'].values.tolist()
for i in range(0,data_v1.shape[0]):
    if not(list_v1[i] + '_v2'in list_v2 and list_v1[i] + '_v3'in list_v3):
        delete_list.append(i)

In [48]:
data_v1.drop(delete_list ,inplace=True)
data_v1.reset_index(drop=True, inplace=True)
list_v1 = data_v1['subjectid'].values.tolist()

In [49]:
list_v2 = []
list_v3 = []
for i in range(0,len(list_v1)):
    list_v2.append(list_v1[i] + '_v2')
    list_v3.append(list_v1[i] + '_v3')

In [50]:
delete_list_2 = []
delete_list_3 = []
for i in range(0,data_v2.shape[0]):
    if not(data_v2['subjectid'][i] in list_v2):
        delete_list_2.append(i)
for i in range(0,data_v3.shape[0]):
    if not(data_v3['subjectid'][i] in list_v3):
        delete_list_3.append(i)

In [51]:
data_v2.drop(delete_list_2 ,inplace=True)
data_v3.drop(delete_list_3 ,inplace=True)
data_v2.reset_index(drop=True, inplace=True)
data_v3.reset_index(drop=True, inplace=True)

In [52]:
list_amyloid = []
for i in range(0,data_v3.shape[0]):
    if data_v3['pib_index'][i] > 1.18:  # list of amyloid positive subjects after visit 3
        list_amyloid.append(data_v1['subjectid'][i] + '_v3')
        print(data_v1['subjectid'][i])

In [54]:
#Store 3 lists into csv
data_v2.to_csv("data_v2.csv", index=False)
data_v1.to_csv("data_v1.csv", index=False)
data_v3.to_csv("data_v3.csv", index=False)

In [56]:
list_v1 = data_v1['subjectid'].values.tolist()#list of subjectid
list_v2 = data_v2['subjectid'].values.tolist()
list_v3 = data_v3['subjectid'].values.tolist()

In [57]:
l1 = []
l2 = []
for i in range(0,len(list_v1)):
    l1.append(list_v1[i]+ "_connectivity_matrix_manhattan.csv")
    l2.append(list_v1[i]+ "_v2_connectivity_matrix_manhattan.csv")

In [58]:
import os
filePath = './v1/'
filelist = os.listdir(filePath)

In [59]:
#clean connectivity_matrix files in folder, leaving only the files related to slected subjects
for i in range(0,len(filelist)):
    if not(filelist[i] in l1):
        if os.path.exists(filePath+ filelist[i]):
            os.remove(filePath+ filelist[i])
        else:
            print ('no' + filePath+ filelist[i])

# 2. Data preparation for APOE group analysis

In [61]:
da = pd.read_csv('./ALL_reggie_sex_apoe_age.csv') 

In [62]:
list_all = [] # list of all subject ids
list_pos = [] # list of apoe+ subject ids
list_neg = [] # list of apoe- subject ids
lis_female = [] # list of female subject ids (for statistic purpose)
lis_age = [] # list of all age information (for statistic purpose)
for i in range(0,da.shape[0]):
    if da['subjectid'][i] in list_v3:
        if da['apoe_e1'][i] == 4 or da['apoe_e2'][i] == 4: # apoe+/- standard
            list_pos.append(da['subjectid'][i])
        else:
            list_neg.append(da['subjectid'][i])
        list_all.append(da['subjectid'][i])
        lis_age.append(da['PIB_Age'][i])
        if da['gender'][i] == 'Female':
            lis_female.append( da['gender'][i])
        #print(da['subjectid'][i])

In [64]:
# store apoe+ subject ids in txt file
with open('APOE_POS.txt', 'w') as filehandle:
    for listitem in list_pos:
        filehandle.write('%s\n' % listitem)

In [65]:
#len(list_neg)

In [66]:
#len(list_all)

In [67]:
#len(lis_female)

In [68]:
#np.std(lis_age)

In [69]:
#np.mean(lis_age)

In [74]:
# store ages for all subject ids in txt file
with open('age.txt', 'w') as filehandle:
    for listitem in lis_age:
        filehandle.write('%s\n' % listitem)

In [71]:
with open('Amyloid_POS.txt', 'w') as filehandle:
    for listitem in list_amyloid:
        filehandle.write('%s\n' % listitem)